In [7]:
import numpy as np
import matplotlib.pyplot as pls
import scipy.optimize as opt

#Implementirung der Funktionen für die Auswertung

## Die Peak sollen mit der Gaussverteilung angefitten werde, diese wird hier implementier, wie ein wrappen für automatische fits

In [8]:
def gauss(x,a,mu,sigma):
    return (a/(np.power(2*3.14,0.5)*sigma))*np.exp(-(x-mu)**2/(2.*sigma**2))

In [11]:
def red_chi_gauss(data, model):
    chisq = np.sum(np.square(data-model)/np.power(data,0.5))
    nu = data.size-4    # -4: -1 for the equation and -3 from the degrees of freedom
    return chisq/nu

In [12]:
def fit_gauss(x,data,guess=[0,0,0]):
    '''
        returns the parameter from the fit, the standard deviation and the reduced chisquare
    '''
    para, cov = opt.curve_fit(gauss,x,np.sqrt(data),guess,maxfev=10000)
    red_chi = red_chi_gauss(data,gauss(x,para[0],para[1],para[2]))
    return para, np.sqrt(np.diag(cov)), red_chi

#Einlesen und vorbereiten der Daten